# OpenAI Agents SDK Demo

OpenAI have released an [Agents SDK](https://openai.github.io/openai-agents-python/) and [Support MCP](https://openai.github.io/openai-agents-python/mcp/). This demo want to show how to use the SDK to make an application that call multiple agents to do a task.

Install the dependencies library:

In [1]:
%pip install -qU openai-agents==0.0.7 python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Stetup LLM and Agent

Setup OpenAI API url, model and key:

In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

OPENAI_MODEL

'qwen2.5:7b'

Enter your OpenAI API key:  ··········


Setup an agent:

In [6]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI

model= OpenAIChatCompletionsModel(
    model=OPENAI_MODEL,
    openai_client=AsyncOpenAI(base_url=OPENAI_API_BASE,api_key=OPENAI_API_KEY),
)

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=model,
)

## Running an Agent

In [7]:
result = await Runner.run(
    starting_agent=agent,
    input="tell me a short story",
)
result.final_output

'Sure, here’s a short story for you:\n\n---\n\nOnce upon a time in a small village nestled between rolling hills and dense forests stood an old wooden well. The villagers had long forgotten its name but knew it as the Heartwell because tales of its magic ran through their community like bloodlines.\n\nOne summer, when the sun scorched the land and the river levels dropped to alarming lows, no amount of rain or water from the spring could stop the village from suffering. The well, old and often neglected, began to whisper with purpose one night during a particularly hot evening.\n\nA young girl named Elara, who had always believed in magic, decided to investigate. She found a small pebble at the bottom, carved with a faded message: "Drink and share." With trembling hands, she took a sip and instantly felt a flood of warmth spreading through her body. Her thirst was quenched, but more than that, she felt an overwhelming sense of happiness and strength.\n\nThe following morning, Elara ret

we create a `RunResultStreaming` object by calling `Runner.run_streamed(...)`, we then asynchronously iterate through the streamed events returned by our LLM using the `response.stream_events()` method:

In [8]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="how are you",
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x11afa3c50>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='__fake_id__', created_at=1743432680.115884, error=None, incomplete_details=None, instructions=None, metadata=None, model='qwen2.5:7b', object='response', output=[], parallel_tool_calls=False, temperature=None, tool_choice='auto', tools=[], top_p=None, max_output_tokens=None, previous_response_id=None, reasoning=

We need to use event.type and event.data to filter out the events we need to progressively display the results returned by LLM.

In [9]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    starting_agent=agent,
    input="tell me a short story",
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Sure, here's a short story for you:

Once upon a time in the bustling city of Newbridge, there lived an old watchmaker named Eli. He had inherited his trade from his father and was known throughout the town for crafting the finest clocks that ran like a well-tuned metronome. However, as years passed, the young generation preferred digital watches over the grandfather clock.

One rainy afternoon, a small girl with large, curious eyes wandered into Eli's shop. Her name was Lily, and she had just moved to Newbridge with her family when they needed repairs for their old house. She noticed an older woman busy repairing a beautiful but tarnished cuckoo clock.

“Hello,” said Lily timidly as she stepped closer. The watchmaker looked up, recognizing the child’s delicate demeanor from stories he often heard in town.

“You must be new here, aren’t you?” Eli asked kindly.

Lily nodded eagerly, “I have. My grandma told me about your amazing clocks.”

Eli smiled warmly and placed a chisel down upon 

## Tools

### Function tools

We can use a `@function_tool` decorator to define a tunction tools:

In [10]:
from agents import function_tool

@function_tool
def add_tool(x: float, y: float) -> float:
    """Add X and Y to get the exact result.
    """
    return x + y

When initializing the `Agent`'s object, we can specify a list of tools via the `tools` parameter:

In [33]:
agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=model,
    tools=[add_tool]
)

Next we look at the results of the execution:

In [12]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="What is the result of 9.776 plus 8.625?",
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x11afa3c50>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='add_tool', description='Add X and Y to get the exact result.', params_json_schema={'properties': {'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}, 'required': ['x', 'y'], 'title': 'add_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11af92ca0>, strict_json_schema=True)], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_ch

Unlike before, we see more different values in type, and also different types appear in event.data, so we filter and display them as well:

In [13]:
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
    ResponseTextDeltaEvent,  # text streaming
)

response = Runner.run_streamed(
    starting_agent=agent,
    input="What is the result of 9.776 plus 8.625?",
)

async for event in response.stream_events():
    if event.type == "raw_response_event": 
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            # this is streamed parameters for our tool call
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            # this is streamed final answer tokens
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        # this tells us which agent is currently in use
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        # these are events containing info that we'd typically
        # stream out to a user or some downstream process
        if event.name == "tool_called":
            # this is the collection of our _full_ tool call after our tool
            # tokens have all been streamed
            print()
            print(f"> Tool Called, name: {event.item.raw_item.name}")
            print(f"> Tool Called, args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            # this is the response from our tool execution
            print(f"> Tool Output: {event.item.raw_item['output']}")

> Current Agent: Assistant
{"x":9.776,"y":8.625}
> Tool Called, name: add_tool
> Tool Called, args: {"x":9.776,"y":8.625}
> Tool Output: 18.401
The result of 9.776 plus 8.625 is 18.401.

## Guardrails

You can define an Agent to do guardrails on input and output. 

In [14]:

from pydantic import BaseModel

# define structure of output for any guardrail agents
class GuardrailOutput(BaseModel):
    is_triggered: bool
    reasoning: str

# define an agent that checks if user is asking about political opinions
politics_agent = Agent(
    name="Politics check",
    instructions="Check if the user is asking you about political opinions",
    output_type=GuardrailOutput,
    model=model,
)

Since it's an Agent, we can use it directly: 

In [15]:

query = "What do you think of the Trudeau-led Liberals?"

result = await Runner.run(
    starting_agent=politics_agent, 
    input=query
    )
result.final_output

GuardrailOutput(is_triggered=True, reasoning='The user asked for an opinion about a specific political party and its leader, which indicates they are seeking a stance or interpretation related to politics.')

You can use the `@input_guardrail` decorator to define an input guardrail:

In [16]:

from agents import (
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail
)

# this is the guardrail function that returns GuardrailFunctionOutput object
@input_guardrail
async def politics_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str,
) -> GuardrailFunctionOutput:
    # run agent to check if guardrail is triggered
    response = await Runner.run(starting_agent=politics_agent, input=input)
    # format response into GuardrailFunctionOutput
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered,
    )

Now we can initialize our normal agent with the input_guardrails parameter:

In [27]:
agent = Agent(
    name="Assistant",
    instructions=(
        "You're a helpful assistant"
    ),
    model=model,
    tools=[add_tool],
    input_guardrails=[politics_guardrail],  # note this is a list of guardrails
)

Let's try a normal input:

In [28]:
result = await Runner.run(
    starting_agent=agent,
    input="What is the result of 9.776 plus 8.625?",
)

result.final_output

'The result of 9.776 plus 8.625 is 18.401.'

Let's look at an example of a protected input that will trigger the guardrail:

In [19]:
result = await Runner.run(
    starting_agent=agent,
    input="What do you think of the Trudeau-led Liberals?",
)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Conversational Agents

Most of the time, we need to have multiple rounds of conversations with the user, and the SDK provides a convenient way to help us do this:

In [34]:
result = await Runner.run(
    starting_agent=agent,
    input="What is the result of 9.776 plus 8.625?"
)
result.final_output

'The result of 9.776 plus 8.625 is 18.401.'

We can use the .to_input_list() method to format the result into a list for the next input:

In [35]:
result.to_input_list()

[{'content': 'What is the result of 9.776 plus 8.625?', 'role': 'user'},
 {'arguments': '{"x":9.776,"y":8.625}',
  'call_id': 'call_uc2bs9zc',
  'name': 'add_tool',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_uc2bs9zc',
  'output': '18.401',
  'type': 'function_call_output'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': 'The result of 9.776 plus 8.625 is 18.401.',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

At this point we can merge this result into the next request:

In [36]:

result = await Runner.run(
    starting_agent=agent,
    input=result.to_input_list() + [
        {"role": "user", "content": "Add the last number by 103.892"}
    ]
)
result.final_output

'The result of adding 18.401 to 103.892 is approximately 122.293.'